<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/ejemplo_regresion_ciencia_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero

#### Ejemplo


# 1 Carga
Partimos de datos de popularidad de canciones en Spotify

In [ ]:
import pandas as pd
# Configuración global para evitar notación científica
pd.set_option('display.float_format', '{:.0f}'.format)

url = "https://github.com/RafaelCaballero/Julio24/raw/main/data/spoty24.csv.zip"
df_raw = pd.read_csv(url,encoding="latin1")
df_raw

In [ ]:
df_raw.info()

Una mala noticia: a partir de ahora Spotify dejará de dar el dato "Spotify Streams" (y todos los demás datos de Spotify). Queremos deducir esta columna a partir del resto (columna 'YouTube Views' en adelante), que son las que parece que sí vamos a poder seguir consiguiendo.

# 2 Análisis exploratorio, preprocesado

## 2.1 Selección de columnas, Tipos
 Una cosa que observamos es que los números los ha leído mal por culpa del separador ",". Vamos a corregir esto:

In [ ]:
url = "https://github.com/RafaelCaballero/Julio24/raw/main/data/spoty24.csv.zip"
df_raw = pd.read_csv(url,encoding="latin1",thousands=",")
df_raw

Nos quedamos solo con las columnas relavantes para nuestro problema

In [ ]:
print(df_raw.columns)
columnas = ['Spotify Streams', 'YouTube Views', 'YouTube Likes', 'TikTok Posts',
       'TikTok Likes', 'TikTok Views', 'YouTube Playlist Reach',
       'Apple Music Playlist Count', 'AirPlay Spins', 'SiriusXM Spins',
       'Deezer Playlist Count', 'Deezer Playlist Reach',
       'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations',
       'Soundcloud Streams', 'Shazam Counts',
       'Explicit Track']

df = df_raw[columnas]
df

In [ ]:
df.info()

En general vemos que las columnas no son detectadas como enteros sino como float. Esto no es grave, pero puede que nos interese que aparezcan como enteros

In [ ]:
for c in df.columns:
  df[c] = df[c].round().astype('Int64')

df.info()

## 2.2 Estadísticas básicas

In [ ]:
df.describe()

Como son tan números tan grandes, podemos dividir por 1000 la etiqueta

In [ ]:
df.loc[:,"Spotify Streams"] = df["Spotify Streams"] // 1000
df.describe()

Nulos:

In [ ]:
df.isnull().sum()

In [ ]:
import missingno as msno
msno.bar(df)

In [ ]:
msno.matrix(df)

In [ ]:
msno.heatmap(df)

Una decisión discutible, pero para hacer pruebas rápidas:

- Quitar primero las columnas  Soundcloud Streams  y SiriusXM Spins

- Después quitar nulos en general

Sin embargo lo posponemos al final

Histogramas sencillos:


In [ ]:
import matplotlib.pyplot as plt

for c in df.columns:
  df[c].hist(bins=50)
  plt.title(c)
  plt.show()


Son las llamadas gráfilas "libres de escala" o "lognormal"; si esasí se deberían ver similares a normales al poner una escala logarítmica en el eje x. Por fortuna la librería seaborn ya hace eso:

In [ ]:
import seaborn as sns

for c in df.columns[:-1]: # quito la col "explicit"
  sns.histplot(df[c], bins=50, log_scale=True)
  plt.title(c)
  plt.show()

Las cols playlist count no parecen lognormal, el resto sí. Una característica de estos valores es que cualquier valor alto se considera outlier:



In [ ]:
import matplotlib.pyplot as plt

for c in df.columns:
  plt.boxplot(df[c].dropna())
  plt.title(c)
  plt.show()

Correlaciones

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.clustermap(df.corr(),
                   method = 'complete',
                   cmap   = 'RdBu',
                   annot  = True,
                   annot_kws = {'size': 8})

Parece que sí hay cierta correlación, la regresión lineal podría valer

Ahora sí quitamos los nulos, y repetimos correlaciones por ver si cambia muy drásticamente

In [ ]:
df2 = df.drop(columns=["Soundcloud Streams","SiriusXM Spins"]).dropna().reset_index(drop=True)
g = sns.clustermap(df2.corr(),
                   method = 'complete',
                   cmap   = 'RdBu',
                   annot  = True,
                   annot_kws = {'size': 8})

# 3 Machine Learning

In [ ]:
yColumn = "Spotify Streams"
XColumns = [ c for c in df2.columns if c!=yColumn] # todas menos la etiqueta

X = df2[XColumns]
y = df2[yColumn]

In [ ]:
import math
from  sklearn.model_selection import cross_val_score

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold

from sklearn.linear_model import LinearRegression




repite = RepeatedKFold(n_splits=20, n_repeats=10)
scorer = make_scorer(mean_squared_error)

metodo = LinearRegression()

res = cross_val_score(metodo,X,y,scoring=scorer,cv=repite)
RMSE = math.sqrt(res.mean())
print(RMSE)

In [ ]:
from sklearn.ensemble import  RandomForestRegressor
metodo = RandomForestRegressor() # RandomForestRegressor

repite = RepeatedKFold(n_splits=20, n_repeats=2)
res = cross_val_score(metodo,X,y,scoring=scorer,cv=repite)
RMSE = math.sqrt(res.mean())
print(RMSE)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
metodo = GradientBoostingRegressor()

res = cross_val_score(metodo,X,y,scoring=scorer,cv=repite)
RMSE = math.sqrt(res.mean())
print(RMSE)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
metodo = AdaBoostRegressor()
res = cross_val_score(metodo,X,y,scoring=scorer,cv=repite)
RMSE = math.sqrt(res.mean())
print(RMSE)

In [ ]:
from sklearn.model_selection import cross_val_predict
metodo = LinearRegression()

y_pred = cross_val_predict(metodo, X, y, cv=len(X))

In [ ]:
import matplotlib.pyplot as plt


x = range(len(y))
fig, ax = plt.subplots(figsize=(200, 5))
ci = 1.96*RMSE
for i in range(len(y_pred)):
    plt.plot([x[i],x[i]], [y_pred[i],y[i]],color="blue",alpha=.4)
ax.fill_between(x, ( y_pred-ci), ( y_pred+ci), color='b', alpha=.1)
ax.scatter(x,y_pred,color="red",s=8,label="predicho")
ax.scatter(x,y,color="green",s=8,label="real")
plt.legend()
plt.show()
fig.savefig("predicciones.pdf",dpi=300)

In [ ]:
y